In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
# Import and read the charity_data.csv
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'
application_df = application_df.drop(["EIN", "NAME"], axis=1)

In [4]:
# Determine the number of unique values in each column
unique_counts = application_df.nunique()
print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df["APPLICATION_TYPE"].value_counts()
print(app_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
cutoff = 500
application_types_to_replace = list(app_counts[app_counts < cutoff].index)

In [7]:
# Replace in dataframe
for app in application_types_to_replace:
  application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app, "Other")

In [8]:
# Check to make sure binning was successful
print(application_df["APPLICATION_TYPE"].value_counts())

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


In [9]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df["CLASSIFICATION"].value_counts()
print(class_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
cutoff = 1000
classifications_to_replace = list(class_counts[class_counts < cutoff].index)

In [11]:
# Replace in dataframe
for cls in classifications_to_replace:
  application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls, "Other")

In [12]:
# Check to make sure binning was successful
print(application_df["CLASSIFICATION"].value_counts())

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [13]:
# Convert categorical data to numeric with pd.get_dummies
encoded_df = pd.get_dummies(application_df)

In [14]:
# Split our preprocessed data into our features and target arrays
X = encoded_df.drop("IS_SUCCESSFUL", axis=1).values
y = encoded_df["IS_SUCCESSFUL"].values

In [15]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

In [17]:
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

In [18]:
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Define the model - deep neural net
nn = tf.keras.models.Sequential()

In [20]:
# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=43))

In [21]:
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

In [22]:
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [23]:
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5734 - accuracy: 0.7211
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5552 - accuracy: 0.7304
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7314
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5508 - accuracy: 0.7326
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5502 - accuracy: 0.7317
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5493 - accuracy: 0.7334
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5480 - accuracy: 0.7358
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7341
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accuracy: 0.7347
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accura

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5558 - accuracy: 0.7247 - 524ms/epoch - 2ms/step
Loss: 0.555761992931366, Accuracy: 0.7246647477149963


In [27]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")